In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from activ.pipeline import score_clusters
import matplotlib.pyplot as plt
import scipy
import h5py
from scipy.stats import entropy
from scipy.spatial.distance import pdist, squareform
import scipy.cluster.hierarchy as _sch
from activ import load_data
from activ.readfile import load_preprocessed, TrackTBIFile, encode, impute
import pandas as pd
from math import ceil
from matplotlib import cm
from sklearn import preprocessing

/Users/ahyeon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/ahyeon/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [8]:
data = load_data()
data.outcome_features

array(['GOSE_NeedAssistance3M', 'GOSE_WorkResume3M',
       'GOSE_DisruptRelation3M', 'GOSE_OtherIssues3M',
       'GOSE_OverallScore3M', 'NeuroPhysHeadache3mo',
       'NeuroPhysBalanceProbl3mo', 'NeuroPhysDizziness3mo',
       'NeuroPhysVisualProbl3mo', 'NeuroPhysFatigue3mo',
       'NeuroPhysLightSensitivity3mo', 'NeuroPhysNoiseSensitivity3mo',
       'NeuroPhysNumbnessTingling3mo', 'NeuroSleepDrowsiness3mo',
       'NeuroSleepSleepingLess3mo', 'NeuroSleepSleepingMore3mo',
       'NeuroSleepTroubleFallingAsleep3mo', 'NeuroCognitiveFoggy3mo',
       'NeuroCognitiveSlowedDown3mo',
       'NeuroCognitiveDiffConcentrating3mo',
       'NeuroCognitiveDiffRemembering3mo',
       'NeuroEmotionalMoreEmotional3mo', 'NeuroEmotionalNervousness3mo',
       'NeuroOverallRating3mo', 'GOSE_Reponder6M_Patient.alone',
       'GOSE_Reponder6M_Patient.plus.relative', 'GOSE_NeedAssistance6M',
       'GOSE_Travel6M', 'GOSE_SocialResume6M', 'GOSE_OtherIssues6M',
       'GOSE_Epilepsy6M', 'GOSE_EpilepsyRis

In [1]:
data = load_preprocessed()
data.isnull().sum().sum()/np.product(data.shape)

NameError: name 'load_preprocessed' is not defined

In [180]:
kept_rows = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/discarded_rows.txt', header=None)
discarded_cols = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/discarded_cols.txt', header=None)
kept_rows = np.hstack(kept_rows.values)
discarded_cols = np.hstack(discarded_cols.values)
kept_rows.shape, discarded_cols.shape


((247,), (444,))

In [181]:
mask = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/mask_feature_outcome.txt', header=None)
mask = mask.values
oc_mask = np.where(mask==1)[0]

In [182]:
a = np.arange(0,586)
b = np.arange(0,918)
discarded_rows = list(set(a).difference(kept_rows))
kept_cols = list(set(b).difference(discarded_cols))

In [183]:
len(discarded_rows), len(discarded_cols), len(kept_rows), len(kept_cols)

(339, 444, 247, 474)

In [184]:
dropped_data = data.drop(data.columns[discarded_cols], axis=1)

In [185]:
dropped_data = data.drop(data.index[discarded_rows])

In [186]:
new_oc_mask = list(set(oc_mask).intersection(kept_cols))

In [187]:
data = data.iloc[:, new_oc_mask]
# data = data.drop(data.index[discarded_rows])
data.shape

(586, 216)

In [191]:
data

,CVLTCuedRecallIntrusionsRaw_6mo,FIMEating_6mo,FIMGrooming_6mo,FIMBathing_6mo,FIMDressingUpperBody_6mo,FIMDressingLowerBody_6mo,FIMToileting_6mo,FIMBladder_6mo,FIMBowelManagement_6mo,FIMBedChairWheelchair_6mo,...,SWLSChangeNothing_6mo,SWLSTotalScore_6mo,CHARTSFAssistPaidHours_6mo,CHARTSFPaidJobHours_6mo,CHARTSFRecreationHours_6mo,CHARTSFSpouse_6mo,CHARTSFRelatives_6mo,CHARTSFIncome_6mo,CHARTSFMedicalCareExpenses_6mo,CHARTSFSelfSufficientTotal_6mo
BR-1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BR-1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BR-1004,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,19.0,0.0,0.0,21.0,No,0.0,22500.0,1750.0,95.27
BR-1006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BR-1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BR-1008,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,26.0,0.0,0.0,3.0,Yes,0.0,30000.0,15000.0,50.98
BR-1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BR-1010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BR-1011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BR-1012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
165/216

0.7638888888888888

In [204]:
data = data[data.isnull().sum(axis=1) < 165]

In [205]:
temp = data
temp.isnull().sum().sum()/np.product(temp.shape)

0.316820987654321

In [207]:
data = impute(data)

In [3]:
mask = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/mask_feature_outcome.txt')
mask = mask['0'].values

In [12]:
patients = open('/Users/ahyeon/Desktop/TBIanalysis/data/patient_id.txt').read().split()

In [34]:
outcome = open('/Users/ahyeon/Desktop/TBIanalysis/data/data_matrix_subset_outcome.txt').read()

In [13]:
fulldata.rename( columns={'Unnamed: 0':'id'}, inplace=True )

In [39]:
data = load_data()
nmfdata = load_data(uoinmf=True)
oc_features = data.outcome_features
bm_features = data.biomarker_features

In [42]:
data.outcomes

array([[1. , 2. , 1. , ..., 0. , 2.3, 0.5],
       [1. , 2. , 1. , ..., 1. , 2. , 0.5],
       [1. , 2. , 1. , ..., 0.5, 4. , 1. ],
       ...,
       [1. , 1. , 2. , ..., 0. , 4. , 1. ],
       [1. , 2. , 1. , ..., 0.5, 4. , 1.5],
       [1. , 2. , 1. , ..., 0. , 4. , 1. ]])

In [44]:
fulldata.

SyntaxError: invalid syntax (<ipython-input-44-9d33caeb1b3e>, line 1)

In [40]:
oc_features

array(['GOSE_NeedAssistance3M', 'GOSE_WorkResume3M',
       'GOSE_DisruptRelation3M', 'GOSE_OtherIssues3M',
       'GOSE_OverallScore3M', 'NeuroPhysHeadache3mo',
       'NeuroPhysBalanceProbl3mo', 'NeuroPhysDizziness3mo',
       'NeuroPhysVisualProbl3mo', 'NeuroPhysFatigue3mo',
       'NeuroPhysLightSensitivity3mo', 'NeuroPhysNoiseSensitivity3mo',
       'NeuroPhysNumbnessTingling3mo', 'NeuroSleepDrowsiness3mo',
       'NeuroSleepSleepingLess3mo', 'NeuroSleepSleepingMore3mo',
       'NeuroSleepTroubleFallingAsleep3mo', 'NeuroCognitiveFoggy3mo',
       'NeuroCognitiveSlowedDown3mo',
       'NeuroCognitiveDiffConcentrating3mo',
       'NeuroCognitiveDiffRemembering3mo',
       'NeuroEmotionalMoreEmotional3mo', 'NeuroEmotionalNervousness3mo',
       'NeuroOverallRating3mo', 'GOSE_Reponder6M_Patient.alone',
       'GOSE_Reponder6M_Patient.plus.relative', 'GOSE_NeedAssistance6M',
       'GOSE_Travel6M', 'GOSE_SocialResume6M', 'GOSE_OtherIssues6M',
       'GOSE_Epilepsy6M', 'GOSE_EpilepsyRis

In [8]:
len(bm_features)

235

In [9]:
len(list(set(fulldatafeatures).intersection(bm_features)))

111

In [17]:
len(patients)

247

In [20]:
fulldata.columns

Index(['id', 'Age', 'Sex', 'RaceAmericanIndian', 'RaceAlaskan', 'RaceAmAl',
       'RaceAsian', 'RaceBlack', 'RaceHAPI', 'RaceWhite',
       ...
       'ICHponsright', 'ICHponsleft', 'ICHmedullaright', 'ICHmedullaleft',
       'ICHcerebellumright', 'ICHcerebellumleft', 'MidlineShiftmm',
       'cisternalcompressionSeverity', 'Fourthventricleshift',
       'Thirdventricleobliteration'],
      dtype='object', length=919)

In [29]:
fulldata[:3]

,id,Age,Sex,RaceAmericanIndian,RaceAlaskan,RaceAmAl,RaceAsian,RaceBlack,RaceHAPI,RaceWhite,...,ICHponsright,ICHponsleft,ICHmedullaright,ICHmedullaleft,ICHcerebellumright,ICHcerebellumleft,MidlineShiftmm,cisternalcompressionSeverity,Fourthventricleshift,Thirdventricleobliteration
0,BR-1001,18,Male,No,No,No,No,No,No,Yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BR-1003,49,Male,No,No,No,No,No,No,Yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BR-1004,24,Male,No,No,No,No,Yes,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
patients = []
for r in range(fulldata.shape[0]):
    for o,feat in enumerate(oc_features):
        if fulldata.feat != 'NA':
            patients.append(r)

In [38]:
len(np.unique(patients))

586

In [41]:
for ii,x in enumerate(fulldata.columns):
    if x == 'GOSE_NeedAssistance3M':
        print(ii)

332


In [31]:
# features with NA > 500 
fulldata.columns[fulldata.isnull().sum() > 500]

Index(['SesPrimChild', 'SesEduNoFather', 'SesEduNoMother', 'SesEduTypeFather',
       'SesEduTypeMother', 'InjSafetyAirbag', 'InjSafetySeatbelt',
       'GcsEDArrPupilReactL', 'EDOsmo', 'NeuroProcedureNumbers', 'ICUReadmit',
       'NumInj_to_AbdPelvis', 'DeathTimeSinceInj', 'DeathCause',
       'DeathCauseOther', 'GOSE_NeedFreqHelp3M', 'GOSE_AssistanceBaseline3M',
       'GOSE_NeedFreqHelp6M', 'GOSE_AssistanceBaseline6M',
       'GOSE_NeedFreqHelp12M', 'GOSE_AssistanceBaseline12M',
       'GOSE_WorkRestriction12M', 'GOSE_WorkBaseline12M',
       'GOSE_SocialRestriction12M', 'GOSE_SocialBaseline12M',
       'GOSE_DisrupExtent12M', 'GOSE_DisruptBaseline12M'],
      dtype='object')

In [37]:
# features with NA > 300 
list(fulldata.columns[fulldata.isnull().sum() > 200])

['SesPrimChild',
 'SesEduNoFather',
 'SesEduNoMother',
 'SesEduTypeFather',
 'SesEduTypeMother',
 'TBILoc',
 'TBILocMemoryGap',
 'InjSafetyAirbag',
 'InjSafetySeatbelt',
 'PresFHospTimeSinceInj',
 'EDArrTemp',
 'GcsEDPupilSizeR',
 'GcsEDArrPupilReactR',
 'GcsEDPupilSizeL',
 'GcsEDArrPupilReactL',
 'EDDischTemp',
 'GcsEDDischPupils',
 'EDOsmo',
 'EDaPtt',
 'EDCO2',
 'EDBunSI',
 'bldsampleTimePostInj',
 'bldsampleMinutesPostInj',
 'GFAP_24H',
 'UCHL1_24H',
 'GFAPAutoAb',
 'T_Tau',
 'P_Tau',
 'P_T_TauRatio',
 'X.4plexGFAP',
 'X.4plexNFL',
 'X.4plexTau',
 'X.4plexUCHL1',
 'X.1plexNFL',
 'pTau181pgmL',
 'nrgn_fred',
 'bdnf_fred',
 'icam_fred',
 'NewGFAP_SOFIA',
 'ED_ETOH',
 'rs6265',
 'rs6277',
 'rs6311',
 'rs17759659',
 'rs3219119',
 'rs11604671',
 'rs4938016',
 'NeuroProcedureNumbers',
 'OtherProcedureInHospital',
 'Numberofproceduresforothersurgeries',
 'ICUAdmTimeSinceInj',
 'ICUDischTimeSinceInj',
 'ICUReadmit',
 'NumInj_to_AbdPelvis',
 'NumInj_to_ChestThorax',
 'NumInj_to_External',
 

In [17]:
[(i, x) for i, x in enumerate(oc_features) if 'GOSE_Overall' in x]

[(4, 'GOSE_OverallScore3M'), (32, 'GOSE_OverallScore6M')]

In [40]:
oc_features

array(['GOSE_NeedAssistance3M', 'GOSE_WorkResume3M',
       'GOSE_DisruptRelation3M', 'GOSE_OtherIssues3M',
       'GOSE_OverallScore3M', 'NeuroPhysHeadache3mo',
       'NeuroPhysBalanceProbl3mo', 'NeuroPhysDizziness3mo',
       'NeuroPhysVisualProbl3mo', 'NeuroPhysFatigue3mo',
       'NeuroPhysLightSensitivity3mo', 'NeuroPhysNoiseSensitivity3mo',
       'NeuroPhysNumbnessTingling3mo', 'NeuroSleepDrowsiness3mo',
       'NeuroSleepSleepingLess3mo', 'NeuroSleepSleepingMore3mo',
       'NeuroSleepTroubleFallingAsleep3mo', 'NeuroCognitiveFoggy3mo',
       'NeuroCognitiveSlowedDown3mo',
       'NeuroCognitiveDiffConcentrating3mo',
       'NeuroCognitiveDiffRemembering3mo',
       'NeuroEmotionalMoreEmotional3mo', 'NeuroEmotionalNervousness3mo',
       'NeuroOverallRating3mo', 'GOSE_Reponder6M_Patient.alone',
       'GOSE_Reponder6M_Patient.plus.relative', 'GOSE_NeedAssistance6M',
       'GOSE_Travel6M', 'GOSE_SocialResume6M', 'GOSE_OtherIssues6M',
       'GOSE_Epilepsy6M', 'GOSE_EpilepsyRis

In [21]:
data.outcomes[:,0]

array([1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 2., 1., 2., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1.

In [8]:
data.outcomes[:,0]

array([1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 2., 1., 2., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1.